## Preparación del notebook

In [ ]:
%pip install kneed

In [ ]:
# Importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score

## Creación de dataset

In [ ]:
#Creación sintética de clusters
features, true_labels = make_blobs(n_samples=300,
                                   centers=3,
                                   cluster_std= 3.5,
                                   random_state=42)

In [ ]:
features[:5]

In [ ]:
true_labels[:5]

## Preprocesamiento de los datos

In [ ]:
# Escalado de características
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [ ]:
scaled_features[:5]

## Algoritmo K-means

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
# Creación de modelo de Kmeans
kmeans = KMeans(init="random",
                n_clusters=3,
                n_init=10,
                max_iter=100,
                random_state=42)

In [ ]:
# Ajuste de algoritmo a caracteristicas escaladas
kmeans.fit(scaled_features)

In [ ]:
# el valor del SSE mas bajo
print("valor del SSE mas bajo: ", kmeans.inertia_)

# Coordenadas de los centroides
print(kmeans.cluster_centers_)

# Numero de interaciones necesarias para converger
print("Numero de iteraciones: ",kmeans.n_iter_)

In [ ]:
# Etiqueta de los datos generadas por Kmeans
kmeans.labels_[:5]

In [ ]:
#Predecir valores de etiquetas
label = kmeans.fit_predict(scaled_features)

print(label)

In [ ]:
#Obtener valores unicos de cluster
u_labels = np.unique(label)

#Diagramar el resultado:
for i in u_labels:
    plt.scatter(scaled_features[label == i,0] , scaled_features[label == i,1] , label = i)
plt.legend()
plt.show()

## Elección del número apropiado de clústeres

### Método del codo

In [ ]:
# Argumentos del algoritmo
kmeans_kwargs = {"init": "random", "n_init": 10, "max_iter": 300, "random_state": 42}
# Lista que almacena los valores de SSE para cada valor K
sse = []
# Ciclo para entrenar algoritmos de kmeans con cada valor de k
for k in range(1, 11):
  kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
  kmeans.fit(scaled_features)
  sse.append(kmeans.inertia_)

In [ ]:
# Visualización de resultados
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.show()

In [ ]:
# Seleccion automatica del numero k
kl = KneeLocator(range(1, 11),
                 sse, curve="convex",
                 direction="decreasing")

kl.elbow

### Coeficiente de silueta

In [ ]:
# Lista contiene los coeficientes de silueta para cada k
silhouette_coefficients = []
# Fíjate que empiezas con 2 grupos para el coeficiente de silueta
for k in range(2, 11):
  kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
  kmeans.fit(scaled_features)
  score = silhouette_score(scaled_features, kmeans.labels_)
  silhouette_coefficients.append(score)

In [ ]:
# Visualización de resultados
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()